# Homework 4

## Description

### Data
[Reuters-21578 Text Categorization Collection Data Set](https://archive.ics.uci.edu/ml/datasets/reuters-21578+text+categorization+collection) - about 2 million instances, 28MB in size


News: 21 SGML files
We only deal with news contents inside <body> </body> tags
The other files will not be needed in this homework

### Format
One text file consisting of lines of records.

Each record contains 9 attributes separated by semicolons: 


Reuters-21578, Distribution 1.0 includes five files (all-exchanges-strings.lc.txt, all-orgs-strings.lc.txt, all-people-strings.lc.txt, all-places-strings.lc.txt, and all-topics-strings.lc.txt) which list the names of *all* legal categories in each set. A sixth file, cat-descriptions_120396.txt gives some additional information on the category sets.


### Task
4 subtasks:
+ (30pt) Given the Reuters-21578 dataset, please calculate the term frequencies, and output the representation of the document contents as a term-document count matrix.
+ (30pt) Implement matrix multiplication by MapReduce. Your program should be able to output the result in appropriate dimensions.
+ (40pt) Given the term-document matrix in (1), compute the SVD decomposition of the matrix using MapReduce. Output the resulting eigenvalues and eigenvectors.
+ (Bonus) Given the term-document matrix in (1), compute the CUR decomposition of the matrix using MapReduce. Output the resulting eigenvalues and eigenvectors.

## Output Format

1. Term-Document Matrix:
A MxN matrix: with rows as term frequencies and columns as documents (N=21,578) 
2. Result of Matrix Multiplication:
A MxR matrix: (MxN) * (NxR)
3. Eigen Pairs:
Eigenvalues sorted in descending order, and their corresponding eigenvectors
4. Eigen Pairs: 
Eigenvalues sorted in descending order, and their corresponding eigenvectors

## Implementation

In [1]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.SparkContext
import org.apache.spark.SparkConf
import org.apache.hadoop.fs._
import org.apache.hadoop.conf.Configuration

import java.io.{File,PrintWriter}

  def printSample(writer: Any, data: Any, title: String = "", format: String = ""){
    println("\n"+title+" Data Sample: " + format)
    println(data+"\n")
  }

  def printSpark(writer: Any, spark: SparkSession): Unit = {
    println("Spark Entity:       " + spark)
    println("Spark version:      " + spark.version)
    println("Spark master:       " + spark.sparkContext.master)
    println("Running 'locally'?: " + spark.sparkContext.isLocal)
    println("")
  }

  def outputWriter(fileString: String): PrintWriter ={
    val outputPath = new Path(fileString)
    val outputStream = outputPath.getFileSystem(new Configuration()).create(outputPath);
    new PrintWriter(outputStream)
  }

  def getFile(fileString: String): Array[String] ={
    val inputPath = new Path(fileString)
    val inputBuffer = scala.collection.mutable.ArrayBuffer.empty[String]
    val iterator = inputPath.getFileSystem(new Configuration()).listFiles(inputPath, false)
    while(iterator.hasNext()){
        val fileStatus = iterator.next()
        if(fileStatus.isFile()){
          inputBuffer += fileStatus.getPath().toString()
        }
    }
    inputBuffer.toArray
  }


val writer = null
printSpark(writer, spark)

Spark Entity:       org.apache.spark.sql.SparkSession@5cf7d99a
Spark version:      2.3.0
Spark master:       local[*]
Running 'locally'?: true



printSample: (writer: Any, data: Any, title: String, format: String)Unit
printSpark: (writer: Any, spark: org.apache.spark.sql.SparkSession)Unit
outputWriter: (fileString: String)java.io.PrintWriter
getFile: (fileString: String)Array[String]
writer: Null = null


In [2]:
val spark2 = spark

var data = getFile("./data")
var sample_data = Array("./data/reut2-017.sgm", "./data/reut2-018.sgm", "./data/reut2-019.sgm")

spark2 = org.apache.spark.sql.SparkSession@5cf7d99a
data = Array(file:/home/micky/school/big_data_mining/hw4/data/reut2-009.sgm, file:/home/micky/school/big_data_mining/hw4/data/reut2-008.sgm, file:/home/micky/school/big_data_mining/hw4/data/reut2-012.sgm, file:/home/micky/school/big_data_mining/hw4/data/reut2-018.sgm, file:/home/micky/school/big_data_mining/hw4/data/reut2-020.sgm, file:/home/micky/school/big_data_mining/hw4/data/reut2-010.sgm, file:/home/micky/school/big_data_mining/hw4/data/reut2-004.sgm, file:/home/micky/school/big_data_mining/hw4/data/reut2-002.sgm, file:/home/micky/school/big_data_mining/hw4/data/reut2-006.sgm, file:/home/micky/school/big_data_mining/hw4/data/reut2-014.sgm, file:/home/micky/school/big_data_mining/hw...


[file:/home/micky/school/big_data_mining/hw4/data/reut2-009.sgm, file:/home/micky/school/big_data_mining/hw4/data/reut2-008.sgm, file:/home/micky/school/big_data_mining/hw4/data/reut2-012.sgm, file:/home/micky/school/big_data_mining/hw4/data/reut2-018.sgm, file:/home/micky/school/big_data_mining/hw4/data/reut2-020.sgm, file:/home/micky/school/big_data_mining/hw4/data/reut2-010.sgm, file:/home/micky/school/big_data_mining/hw4/data/reut2-004.sgm, file:/home/micky/school/big_data_mining/hw4/data/reut2-002.sgm, file:/home/micky/school/big_data_mining/hw4/data/reut2-006.sgm, file:/home/micky/school/big_data_mining/hw4/data/reut2-014.sgm, file:/home/micky/school/big_data_mining/hw4/data/reut2-000.sgm, file:/home/micky/school/big_data_mining/hw4/data/reut2-013.sgm, file:/home/micky/school/big_data_mining/hw4/data/reut2-007.sgm, file:/home/micky/school/big_data_mining/hw4/data/reut2-005.sgm, file:/home/micky/school/big_data_mining/hw4/data/reut2-016.sgm, file:/home/micky/school/big_data_mining

In [3]:
import scala.collection.mutable.ListBuffer

//TODO: Sample -> Actual
var regex = """(?s)<REUTERS.*?NEWID="(\d*)".*?>.*?<TEXT.*?>(.*?)<\/TEXT>.*?<\/REUTERS>""".r
var body_regex = """(?s).*?<BODY.*?>(.*?)<\/BODY>.*?""".r
var newsParse = spark.sparkContext.emptyRDD[(Int, String)]

sample_data.map{
    path  =>
    val singleNewsParse = spark.sparkContext.wholeTextFiles(path).flatMap{
        case (_, string) =>
        val list =  ListBuffer[Tuple2[Int, String]]()
        for(m <- regex.findAllIn(string).matchData){
            for( n <- body_regex.findAllIn(m.group(2)).matchData){
                list += (Tuple2(m.group(1).toInt, n.group(1).replace("\n Reuter\n&#3;","")))
            }
        }
        list.toSeq
    }
    newsParse = newsParse.union(singleNewsParse)
}
var newsCount = newsParse.count()
print("\nCount: "+ newsCount+"\n")
printSample(writer, newsParse.take(2).mkString("\n\n"), "Parsed Sample")


Count: 2749

Parsed Sample Data Sample: 
(17001,The Brazilian Coffee Institute, IBC,
plans to sell in a series of auctions over the next few weeks
robusta coffee purchased in London last year, but details of
where and when auctions will take place are still to be
finalised, IBC president Jorio Dauster told reporters.
    The sales of 630,000 bags of robusta and an unspecified
amount of Brazilian arabica coffee will take place over a
minimum of six months but it is not decided where sales will
take place or whether they will be held weekly or monthly.
    The amount offered at each sale has also not been set, but
could be in the order of 100,000 bags, Dauster said.)

(17002,AMCA International Ltd said it
appointed president and chief executive officer WIlliam Holland
to succeed Kenneth Barclay as chairman.
    Barclay, who is 60 years old, decided not to stand for
reappointment as chairman this year but will continue as a
director, AMCA said.)



regex = (?s)<REUTERS.*?NEWID="(\d*)".*?>.*?<TEXT.*?>(.*?)<\/TEXT>.*?<\/REUTERS>
body_regex = (?s).*?<BODY.*?>(.*?)<\/BODY>.*?
newsParse = UnionRDD[12] at union at <console>:59
newsCount = 2749


2749

### Task 1 - Term-Frequency Matrix

In [4]:
def wordPreProcess(input: Any): Array[String] = {
        var matchRegex = """([$]?(?:[\w]+(?:[\w',]*[\w]+)+|[\w]))""".r
        var list =  ListBuffer[String]()
        for(m <- matchRegex.findAllIn(input.toString.toLowerCase).matchData;
          e <- m.subgroups)
          list+=e
        list.toArray
    }

var newsCount = spark.sparkContext.broadcast(newsParse.count())
var matrixIndex = spark.sparkContext.broadcast(newsParse.map{
    case(id, string) =>
    id.toInt
}.sortBy{
    case (id) => id
}.zipWithIndex.collectAsMap())

printSample(writer, matrixIndex.value.take(5).mkString("\n"), "Ziped With Index Sample")


Ziped With Index Sample Data Sample: 
18063 -> 992
17408 -> 402
18404 -> 1314
17749 -> 711
18090 -> 1013



newsCount = Broadcast(6)
matrixIndex = Broadcast(11)


wordPreProcess: (input: Any)Array[String]


Broadcast(11)

In [5]:
var flattenTerms = newsParse.flatMap{
    case (id, string)  =>
    wordPreProcess(string).map{
        word =>
        ((word, matrixIndex.value(id).toInt), 1)
    }  
}.reduceByKey{
    (x, y) => x + y
}.map{
    case((word, id),count) =>
    (word,(id, count))
}.groupByKey()
printSample(writer, flattenTerms.take(20).mkString("\n"), "Parsed Shingle Sample")


Parsed Shingle Sample Data Sample: 
(francs,CompactBuffer((1271,1), (2342,1), (859,7), (1363,1), (2049,2), (455,1), (2376,2), (139,1), (1741,1), (526,2), (856,8), (57,2), (1890,1), (2319,1), (376,1), (143,3), (2089,2), (1361,1), (1906,4), (695,2), (1467,2), (945,1), (1419,1), (270,1), (1429,8), (1942,4), (944,4), (1432,1), (84,3), (2324,2), (1581,1), (1448,2), (635,1), (2352,5)))
(constraints,CompactBuffer((913,1), (240,1), (837,1), (1907,1), (883,1), (1812,1), (868,1)))
(obliging,CompactBuffer((241,1)))
(brewery,CompactBuffer((1264,1), (123,1)))
(topped,CompactBuffer((1847,1), (856,1), (721,2), (998,1), (2237,1)))
(wla,CompactBuffer((2112,1)))
(datacard,CompactBuffer((2455,9)))
(dole,CompactBuffer((2698,2), (344,2), (79,1), (690,1), (993,3)))
(gowns,CompactBuffer((2575,1)))
(dayton,CompactBuffer((2266,2), (2079,4), (2220,7), (607,1)))
(albertson,CompactBuffer((1714,1)))
(fred,CompactBuffer((2203,1), (353,1), (2138,1), (2037,1)))
(preventing,CompactBuffer((873,1), (2327,1), (2455,1), 

flattenTerms = ShuffledRDD[23] at groupByKey at <console>:55


ShuffledRDD[23] at groupByKey at <console>:55

In [6]:
var termDocumentMatrix = flattenTerms.map{
    case (word, iterable) =>
    var row = Array.fill(newsCount.value.toInt){0}
    iterable.toVector.map{
        case(id, count) =>
        row(id) = count
    }
    row
}

termDocumentMatrix = MapPartitionsRDD[24] at map at <console>:44


MapPartitionsRDD[24] at map at <console>:44

In [7]:
val termDocumentString = termDocumentMatrix.map{
      array =>
      array.mkString(",")
}

termDocumentString.saveAsTextFile("./output/termDocumentString")

termDocumentString = MapPartitionsRDD[25] at map at <console>:46


MapPartitionsRDD[25] at map at <console>:46

### Task 2 - Matrix Multiplication

In [7]:
val rand = scala.util.Random
val r_value = 3
var otherMatrix = spark.sparkContext.parallelize(Array.fill(newsCount.value.toInt,r_value){rand.nextInt(2000) - 1000})

rand = scala.util.Random$@15f1c2d1
r_value = 3
otherMatrix = ParallelCollectionRDD[25] at parallelize at <console>:42


ParallelCollectionRDD[25] at parallelize at <console>:42

In [8]:
import org.apache.spark.rdd.RDD
def matrixMultiplication(matrix1: RDD[Array[Int]], matrix2: RDD[Array[Int]]): RDD[Array[Int]] = {
    var otherMatrix = spark.sparkContext.broadcast(matrix2.zipWithIndex.map{x => (x._2, x._1)}.collectAsMap())
    matrix1.map{
        array =>
        var newRow = Array.fill(otherMatrix.value(0).length){0}
        newRow.zipWithIndex.map{
            case(value, columnid) =>
            var newValue = value
            array.zipWithIndex.foreach{
                case(arrayValue, rowid) =>
                newValue += arrayValue * otherMatrix.value(rowid)(columnid)
            }
            newValue
        }
    }
}

matrixMultiplication: (matrix1: org.apache.spark.rdd.RDD[Array[Int]], matrix2: org.apache.spark.rdd.RDD[Array[Int]])org.apache.spark.rdd.RDD[Array[Int]]


In [9]:
var multipliedMatrix = matrixMultiplication(termDocumentMatrix, otherMatrix)

multipliedMatrix.take(2).foreach{
    array =>
    println(array.mkString(", "))
}

-16610, -9520, 1261
-1182, 695, -1421


multipliedMatrix = MapPartitionsRDD[28] at map at <console>:47


MapPartitionsRDD[28] at map at <console>:47

In [11]:
var mulString = multipliedMatrix.map{
      array =>
      array.mkString(",")
}

mulString.saveAsTextFile("./output/multipliedMatrix")

mulString = MapPartitionsRDD[31] at map at <console>:55


MapPartitionsRDD[31] at map at <console>:55

### Task 3 - SVD Decomposition

In [7]:
import org.apache.spark.rdd.RDD
import math.{sqrt,pow}

def matrixDoubleMultiplication(matrix1: RDD[Array[Double]], matrix2: RDD[Array[Double]]): RDD[Array[Double]] = {
    var otherMatrix = spark.sparkContext.broadcast(matrix2.zipWithIndex.map{x => (x._2, x._1)}.collectAsMap())
    matrix1.map{
        array =>
        var newRow = Array.fill(otherMatrix.value(0).length){0.0}
        newRow.zipWithIndex.map{
            case(value, columnid) =>
            var newValue = value
            array.zipWithIndex.foreach{
                case(arrayValue, rowid) =>
                newValue += arrayValue * otherMatrix.value(rowid)(columnid)
            }
            newValue 
        }
    }
}

def matrixSubtract(matrix1: RDD[Array[Double]], matrix2: RDD[Array[Double]]): RDD[Array[Double]] = {
    var otherMatrix = spark.sparkContext.broadcast(matrix2.zipWithIndex.map{x => (x._2, x._1)}.collectAsMap())
    matrix1.zipWithIndex.map{
        case (array, rowid) =>
        array.zipWithIndex.map{
            case(value, columnid) =>
            value - otherMatrix.value(rowid)(columnid)
        }
    }
}

def matrixTranspose(matrix: RDD[Array[Double]]): RDD[Array[Double]] = {
    var transpose = matrix.collect.toSeq.transpose.map{seq => seq.toArray}
    spark.sparkContext.parallelize(transpose)
}


def matrixAbs(matrix: RDD[Array[Double]]): Double = {
    var sum = matrix.map{
        array => 
        array.map{
            value =>
            pow(value, 2)
        }
    }.reduce((a,b) => Array.fill(1){a(0) + b(0)})
    sqrt(sum(0))
}

matrixDoubleMultiplication: (matrix1: org.apache.spark.rdd.RDD[Array[Double]], matrix2: org.apache.spark.rdd.RDD[Array[Double]])org.apache.spark.rdd.RDD[Array[Double]]
matrixSubtract: (matrix1: org.apache.spark.rdd.RDD[Array[Double]], matrix2: org.apache.spark.rdd.RDD[Array[Double]])org.apache.spark.rdd.RDD[Array[Double]]
matrixTranspose: (matrix: org.apache.spark.rdd.RDD[Array[Double]])org.apache.spark.rdd.RDD[Array[Double]]
matrixAbs: (matrix: org.apache.spark.rdd.RDD[Array[Double]])Double


In [13]:
var count = 0
var doubleTermMatrix = termDocumentMatrix.map{array => array.map{x => x.toDouble}}
var transposeMatrix = matrixTranspose(doubleTermMatrix)
var symmetricMatrix = matrixDoubleMultiplication(doubleTermMatrix, transposeMatrix)

var eigenValue = 0.0

while(count < 5 && !eigenValue.isNaN){  
    var xMatrix = spark.sparkContext.parallelize(Array.fill(symmetricMatrix.count().toInt,1){1.0})
    var prevDiv = 0.0
    var div = 10.0
    var countNum = 0

    while(abs(div-prevDiv) >= 0.00000000000001){
        countNum += 1
        prevDiv = div
        var mul = matrixDoubleMultiplication(symmetricMatrix, xMatrix) 
        div = matrixAbs(xMatrix)

        xMatrix = mul.map{
            array=> 
            array.map{ value => value/div }
        }
    }

    var transposeX = matrixTranspose(xMatrix)
    var partial = matrixDoubleMultiplication(transposeX, symmetricMatrix)
    var eigen = matrixDoubleMultiplication(partial, xMatrix)
    eigenValue = eigen.collect()(0)(0)

    println(countNum +": "+eigenValue)
    xMatrix.map{array => array.mkString(", ")}.saveAsTextFile("./output/SVD/"+eigenValue)

    var partial2 = matrixDoubleMultiplication(xMatrix, transposeX)
    var partial3 = partial2.map{
        array=>
        array.map{ value => value * eigenValue }
    }
    
    symmetricMatrix = matrixSubtract(symmetricMatrix, partial3)
    
    count += 1
}

32: 3.0275189385964336E10
5: -2.543515231731319E52
11: 1.0645638478974878E262
3: NaN


count = 4
doubleTermMatrix = MapPartitionsRDD[1538] at map at <console>:61
transposeMatrix = ParallelCollectionRDD[1539] at parallelize at <console>:69
symmetricMatrix = MapPartitionsRDD[1869] at map at <console>:58
eigenValue = NaN


NaN

103820004 Michael Fu